## **Setup**

In [ ]:
from google.colab import drive
mount_path = '/content/gdrive/'
drive.mount(mount_path)

model_path='My Drive/AI For Good - AI Blitz 3/AutoDrive/Models/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [ ]:
import os
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import pytz
from zipfile import ZipFile
from tempfile import TemporaryDirectory
import requests
import cv2
import shutil
import glob

import tensorflow as tf
from tensorflow import keras

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

# from sklearn.metrics import roc_auc_score, roc_curve, auc
# from sklearn.model_selection import train_test_split

# Load the TensorBoard notebook extension
%load_ext tensorboard

from google.colab.patches import cv2_imshow 

# plot options
# plt.rcParams.update({'font.size': 11})
plt.style.use('fivethirtyeight')

# Data Loading

In [ ]:
# custom copytree because Colab doesn't have the latest version of shutil.copytree which now contains the dirs_exist_ok flag solving this issue
# copied verbatim from https://stackoverflow.com/a/12514470/5991868
def copytree(src, dst, symlinks=False, ignore=None):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)

In [ ]:
# home_dir is the home directory for the images, from which training and validation splits will be taken
home_dir='/content/All_Data/'

# this can take a bit (maybe 30s)
if not os.path.exists(home_dir):
  os.mkdir(home_dir)
  for cview in ['Front','Left','Rear','Right']:
    os.mkdir(os.path.join(home_dir,'camera'+cview))

  urlbase = 'https://datasets.aicrowd.com/default/aicrowd-practice-challenges/public/autodri/v0.1/'

  # for filename in ['train','val']:
  for filename in ['test']:
    # create a temporary directory using TemporaryDirectory and context manager and unzip to there
    with TemporaryDirectory() as tmpdirname:
      # download the file
      requrl = requests.get(f'{urlbase}{filename}.zip')
      with open(f'{tmpdirname}/{filename}.zip', "wb") as zip:
        zip.write(requrl.content)
      # unzip
      with ZipFile(f'{tmpdirname}/{filename}.zip','r') as zip_ref:
        zip_ref.extractall(tmpdirname)
      # copy the data to the All_Data dir
      shutil.copy2(f'{tmpdirname}/{filename}/{filename}.csv',f'{home_dir}/{filename}.csv')
      for cview in ['Front','Left','Rear','Right']:
        copytree(f'{tmpdirname}/{filename}/camera{cview}/',f'{home_dir}/camera{cview}/')


In [ ]:
combdf=pd.read_csv(f'{home_dir}/test.csv')

# creat the img_list column
combdf['img_list']=combdf['filename'].apply(
    lambda fname: [os.path.join(home_dir,f'camera{cview}/{fname}') for cview in ['Left','Front','Right','Rear']])

In [ ]:
combdf.head()

,filename,img_list
0,0.jpg,"[/content/All_Data/cameraLeft/0.jpg, /content/..."
1,1.jpg,"[/content/All_Data/cameraLeft/1.jpg, /content/..."
2,2.jpg,"[/content/All_Data/cameraLeft/2.jpg, /content/..."
3,3.jpg,"[/content/All_Data/cameraLeft/3.jpg, /content/..."
4,4.jpg,"[/content/All_Data/cameraLeft/4.jpg, /content/..."


# Model Load

In [ ]:
loaded=tf.saved_model.load(f'{os.path.join(mount_path,model_path)}200903_0827_AutoDrive_frozen')

In [ ]:
model = keras.models.load_model(f'{os.path.join(mount_path,model_path)}200903_0827_AutoDrive_frozen')

In [ ]:
def get_imgs(imlist):
  imgs=[]
  for impath in imlist:
    img = load_img(impath,target_size=(224, 224))
    imgs.append(img_to_array(img))
    # Pillow images should be closed after `load_img`,
    # but not PIL images.
    if hasattr(img, 'close'):
                img.close()
                
  return np.stack(imgs,axis=0)

def imageseq_generator(df, batch_size = 64):  
  inds=df.index.to_list()
  # while True:
    # shuffle the indices for the epoch
    # np.random.shuffle(inds)

  # Get index to start each batch: [0, batch_size, 2*batch_size, ..., max multiple of batch_size &lt;= num_samples]
  for offset in range(0, len(inds), batch_size):
    # Get the samples you'll use in this batch
    batch_inds = inds[offset:(offset+batch_size)]

    batch_input  = []
    batch_output = []     
    # Read in each input, perform preprocessing and get labels
    for ind in batch_inds:
        batch_input.append(get_imgs(df.loc[ind,'img_list']))
        # batch_output.append(df.loc[ind,'canSteering'])

    # Return a tuple of (input, output) to feed the network    
    # yield (np.array(batch_input), np.array(batch_output))
    yield np.array(batch_input)

In [ ]:
testX=np.array([get_imgs(combdf.loc[0,'img_list']),get_imgs(combdf.loc[1,'img_list'])])
print(testX.shape)

(2, 4, 224, 224, 3)


In [ ]:
preds=model.predict(imageseq_generator(combdf, batch_size = 64),verbose=1)

333/333 [==============================] - 250s 750ms/step


In [ ]:
print(combdf.shape)
print(preds.shape)

(21269, 2)
(21269, 1)


In [ ]:
submitdf=pd.DataFrame(data=preds,columns=['canSteering'])
submitdf['filename']=combdf['filename']
submitdf.head()

,canSteering,filename
0,93.125481,0.jpg
1,107.965385,1.jpg
2,131.888199,2.jpg
3,135.473969,3.jpg
4,104.063980,4.jpg


In [ ]:
submitdf.to_csv(home_dir+'submission.csv',index=True)

In [ ]:
from google.colab import files
files.download(home_dir+'submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
base_model = keras.applications.EfficientNetB4(include_top = False, weights = 'imagenet',
    input_shape = (224, 224, 3))

# freeze the weights of the pre-trained layers
for layer in base_model.layers:
  layer.trainable = False

71688192/71686520 [==============================] - 1s 0us/step


In [ ]:
# create a Sequential model
model = keras.models.Sequential()

# add base_model for 4 input images (keeping the right shape
model.add(keras.layers.TimeDistributed(base_model, input_shape=(4, 224, 224, 3)))

# now, flatten on each output to send 4 outputs with one dimension to LSTM
model.add(keras.layers.TimeDistributed(keras.layers.Flatten()))
model.add(keras.layers.LSTM(256, activation='relu', return_sequences=False))

# finalize with standard MLP

### THESE DON'T HAVE model.add!!!###
# keras.layers.Dense(128, activation=None),
# keras.layers.BatchNormalization(),
# keras.layers.Activation('relu'),
# keras.layers.Dropout(0.25),

# keras.layers.Dense(128, activation=None),
# keras.layers.BatchNormalization(),
# keras.layers.Activation('relu'),
# keras.layers.Dropout(0.25),

# keras.layers.Dense(64, activation=None),
# keras.layers.BatchNormalization(),
# keras.layers.Activation('relu'),
# keras.layers.Dropout(0.25),

model.add(keras.layers.Dense(1,activation='linear'))

### Compile model

In [ ]:
init_lr=1e-2
optimizer = keras.optimizers.Nadam(lr=init_lr)
loss_function = keras.losses.MeanSquaredError()

model.compile(loss=loss_function, optimizer=optimizer)

max_epochs = 100
batch_size = 32

In [ ]:
# early stopping callback
# patience is number of epochs without improvement
early_stopping_cb = keras.callbacks.EarlyStopping(patience=4,restore_best_weights=True)

In [ ]:
# # set up Tensorboard
# logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
# %tensorboard --logdir logs

In [ ]:
# # tensorboard 
# logdir = os.path.join("logs", datetime.datetime.now(pytz.timezone('US/Eastern')).strftime("%y%m%d_%H%M"))
# tb_cb = keras.callbacks.TensorBoard(logdir)

cv_fold=0

train_df=cvdict[cv_fold]['train']
val_df=cvdict[cv_fold]['val']

train_steps=train_df.shape[0]//batch_size
val_steps=val_df.shape[0]//batch_size

# train it!
history = model.fit(imageseq_generator(train_df,batch_size), epochs = 5,
                    validation_data = imageseq_generator(val_df,batch_size),
                    steps_per_epoch = train_steps, validation_steps = val_steps)
                    # callbacks = [early_stopping_cb])

Epoch 1/5
1665/1665 [==============================] - 935s 562ms/step - loss: 33108.6133 - val_loss: 19825.0508
Epoch 2/5
1665/1665 [==============================] - 885s 531ms/step - loss: 14849.2520 - val_loss: 11290.9053
Epoch 3/5
 700/1665 [===========>..................] - ETA: 6:55 - loss: 11072.2686

In [ ]:
datestr=datetime.datetime.now(pytz.timezone('US/Eastern')).strftime("%y%m%d_%H%M")
model.save(f'{os.path.join(mount_path,model_path)}{datestr}_AutoDrive_frozen')

In [ ]:
# unfreeze the weights of the pre-trained layers
for layer in base_model.layers:
  layer.trainable = True

In [ ]:
init_lr=1e-4
optimizer = keras.optimizers.Nadam(lr=init_lr)
loss_function = keras.losses.MeanSquaredError()

model.compile(loss=loss_function, optimizer=optimizer)

max_epochs = 100
batch_size = 128

In [ ]:
# train it!
history = model.fit(imageseq_generator(train_df,batch_size), epochs = 10,
                    validation_data = imageseq_generator(val_df,batch_size),
                    steps_per_epoch = train_steps, validation_steps = val_steps)
                    # callbacks = [early_stopping_cb])

In [ ]:
datestr=datetime.datetime.now(pytz.timezone('US/Eastern')).strftime("%y%m%d_%H%M")
model.save(f'{os.path.join(mount_path,model_path)}{datestr}_AutoDrive_frozen')